In [1]:
import pandas as pd
import numpy as np
import datetime
import pandas.io.sql as pdsql
import matplotlib.pyplot as plt
%matplotlib inline

#!pip install psycopg2
import json
import psycopg2

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [0]:
#connect to the server
conn = psycopg2.connect(host="rds-trafficjam.cpze98eqmmci.us-east-2.rds.amazonaws.com",
	                    database="dbtrafficjam", user="trafficjam", password="anss!!596")

cur = conn.cursor()

For 2018 scaling factor, I'll use the data compiled by Concord Engineering.

In [3]:
# I had difficulty importing a single sheet from the Concord study Excel file so I created a CSV with just the annual study comparison data. It is uploaded in the Google Drive.
# Importing from a local drive is easier than from Google Drive, so that's the approach I've taken here.
# Save the file to your local drive, then run this cell. A box asking you to choose a file will appear.

from google.colab import files
uploaded = files.upload()

Saving 2018_Annual_Study_Concord.csv to 2018_Annual_Study_Concord.csv


In [0]:
import io
df2018 = pd.read_csv(io.BytesIO(uploaded['2018_Annual_Study_Concord.csv']))

In [5]:
len(df2018)

14430

In [6]:
df2018.columns

Index(['Elmntkey', 'Study_Area', 'Sub_Area', 'Full Area Name', 'Study Date',
       'Study_Time', 'Side', 'Unitdesc', 'Peak Hour_SDOT', 'Parking_Spaces',
       'Total_Vehicle_Count', 'Dp_Count', 'Rpz_Count', 'TG_Car2Go', 'BMW_DN',
       'Field Notes', 'Construction', 'Event Closure', 'Subarea Label',
       'Study Year', 'Peak Hour? (Yes or No)', 'RPZ Blocks', 'CSM',
       'Time Stamp', 'Time Stamp Rounded', 'Scratch Concatenate', 'Unique ID',
       'Calculated Occupancy', 'Adjusted Observations', 'Adjusted-Calculated',
       'CalcOcc%', 'ObsOcc%', 'Nonpayment%', 'Total_Occ%'],
      dtype='object')

In [7]:
# Filter to Belltown North
df2018 = df2018[df2018['Full Area Name'] == 'Belltown North']
len(df2018)

1602

In [8]:
len(df2018[df2018['Parking_Spaces'] == 0])

36

In [0]:
# Exclude rows where number of parking spots is 0
df2018 = df2018[df2018['Parking_Spaces'] > 0]

In [0]:
# Calculate scaling/calibration factor
# Number of vehicles actually parked
actual_parking_load = sum(df2018['Total_Vehicle_Count'])
# Number of vehicles parked estimated by transaction data
estimated_parking_load = sum(df2018['Calculated Occupancy'])


In [11]:
# Calculate using actual/estimated counts
actual_parking_load/estimated_parking_load

1.1973099860423804

In [12]:
# Same method, but account for number of spaces, could be applied to rate
total_spaces = sum(df2018['Parking_Spaces'])
(actual_parking_load - estimated_parking_load)/total_spaces + 1

1.1338209982788297

In [13]:
# How many rows have actual and transaction occupancy at 0?
len(df2018[np.logical_and(df2018['Total_Vehicle_Count'] == 0, df2018['Calculated Occupancy'] == 0)])

31

In [14]:
# How many rows have actual occupancy at 0?
len(df2018[df2018['Calculated Occupancy'] == 0])

85

In [15]:
# What percentage of rows have actual occupancy at 0?
len(df2018[df2018['Calculated Occupancy'] == 0])/len(df2018) * 100

5.427841634738186

In [0]:
# Exclude rows with 0 transaction count
df2018_small = df2018[df2018['Calculated Occupancy'] > 0]

In [17]:
# Calculate ratio for each row using counts
df2018_small['count_ratio'] = df2018_small['Total_Vehicle_Count']/df2018_small['Calculated Occupancy']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
# Calculate mean ratio across all rows
np.mean(df2018_small['count_ratio'])

1.3714592169292894

In [0]:
# Calculate difference in occupancy rates for each block for each time then take mean
df2018['rate_diff'] = pd.to_numeric(df2018['ObsOcc%'].str.rstrip('%'))/100 - pd.to_numeric(df2018['CalcOcc%'].str.rstrip('%'))/100

In [20]:
np.mean(df2018['rate_diff']) + 1

1.0776883780332058

In [0]:
# Import 2018 data from database and compare to see if results are different

In [0]:
#connect to the server
conn = psycopg2.connect(host="rds-trafficjam.cpze98eqmmci.us-east-2.rds.amazonaws.com",
	                    database="dbtrafficjam", user="trafficjam", password="anss!!596")

cur = conn.cursor()

In [0]:
df_2018_db = pd.read_sql("SELECT * FROM vw_2018_join", conn)

In [24]:
df_2018_db.columns

Index(['source_element_key', 'main_lane', 'between_one', 'between_second',
       'sideofstreet', 'date_of_transaction', 'mm_transaction',
       'yyyy_transaction', 'dd_transaction', 'dow_transaction',
       'doy_transaction', 'week_transaction', 'time_of_transaction',
       'hour_transaction', 'min_transaction', 'occupied_spots', 'empty_spots',
       'total_spots', 'paidparkingarea', 'paidparkingsubarea',
       'parkingcategory', 'parkingtimelimitcategory', 'latitude', 'longitude',
       'geog', 'segkey', 'total_spots_survey', 'occupied_spots_survey',
       'time_of_survey', 'weather_at_hour', 'average_temp', 'rain_inches',
       'humidity', 'event', 'employment', 'avg_traffic_density',
       'main_city_street', 'business_license_count',
       'parking_lots_garages_count', 'banks_count', 'bars_restaurants_count',
       'event_venues_count', 'grocery_market_count', 'gyms_count',
       'hotels_count', 'medical_offices_count', 'museums_count',
       'salons_count', 'shopping

In [25]:
len(df_2018_db[np.logical_and(df_2018_db['total_spots_survey'] != df_2018_db['total_spots'], ~np.isnan(df_2018_db['total_spots_survey']))])

317

In [0]:
# Remove rows with no survey results
df_2018_db = df_2018_db[~np.isnan(df_2018_db['total_spots_survey'])]

In [27]:
len(df_2018_db)

3211

In [28]:
# Compare to method that doesn't include number of spots first to see effect of time difference
# Concord data gave results of 1.19
sum(df_2018_db['occupied_spots_survey'])/sum(df_2018_db['occupied_spots'])

1.1849461017798948

In [29]:
# Compare to method that does include number of spots to see effect
# Concord data gave results of 1.13
(sum(df_2018_db['occupied_spots_survey']) - sum(df_2018_db['occupied_spots']))/sum(df_2018_db['total_spots']) + 1

1.1238406983087834

In [30]:
df_2018_db.head()

,source_element_key,main_lane,between_one,between_second,sideofstreet,date_of_transaction,mm_transaction,yyyy_transaction,dd_transaction,dow_transaction,...,banks_count,bars_restaurants_count,event_venues_count,grocery_market_count,gyms_count,hotels_count,medical_offices_count,museums_count,salons_count,shopping_count
829,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,0,41,3,4,1,6,8,1,16,14
830,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,0,41,3,4,1,6,8,1,16,14
831,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,0,41,3,4,1,6,8,1,16,14
832,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,0,41,3,4,1,6,8,1,16,14
833,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,0,41,3,4,1,6,8,1,16,14


### Scaling factor grouped by morning/afternoon/evening - our db

In [0]:
def assign_part_of_day(x):
  if np.logical_and(x>=8, x<=11):
    return "Morning"
  elif np.logical_and(x>=12, x<=16):
    return "Afternoon"
  else:
    return "Evening"

In [0]:
df_2018_db['parts_of_day'] = df_2018_db['hour_transaction'].apply(assign_part_of_day)

In [33]:
len(df_2018_db)

3211

In [34]:
df_2018_db.shape

(3211, 50)

In [0]:
df_2018_scaling = pd.DataFrame(df_2018_db.groupby(['parts_of_day'], as_index=False)[['occupied_spots', 'occupied_spots_survey']].sum())

In [36]:
len(df_2018_scaling)

3

In [37]:
df_2018_scaling.head()

,parts_of_day,occupied_spots,occupied_spots_survey
0,Afternoon,7349,8423.0
1,Evening,3944,4451.0
2,Morning,4663,6033.0


In [0]:
df_2018_scaling['scaling_factor'] = (df_2018_scaling['occupied_spots_survey']/df_2018_scaling['occupied_spots'])

In [39]:
df_2018_scaling.head()

,parts_of_day,occupied_spots,occupied_spots_survey,scaling_factor
0,Afternoon,7349,8423.0,1.146142
1,Evening,3944,4451.0,1.128550
2,Morning,4663,6033.0,1.293802


In [0]:
df_2018_new=pd.merge(df_2018_db, df_2018_scaling, how="left", left_on=['parts_of_day'], right_on=['parts_of_day']) 

In [41]:
df_2018_new.head()

,source_element_key,main_lane,between_one,between_second,sideofstreet,date_of_transaction,mm_transaction,yyyy_transaction,dd_transaction,dow_transaction,...,gyms_count,hotels_count,medical_offices_count,museums_count,salons_count,shopping_count,parts_of_day,occupied_spots_y,occupied_spots_survey_y,scaling_factor
0,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,1,6,8,1,16,14,Afternoon,7349,8423.0,1.146142
1,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,1,6,8,1,16,14,Afternoon,7349,8423.0,1.146142
2,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,1,6,8,1,16,14,Morning,4663,6033.0,1.293802
3,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,1,6,8,1,16,14,Afternoon,7349,8423.0,1.146142
4,1025,1ST AVE,BATTERY ST,WALL ST,SW,2018-05-03,5.0,2018.0,3.0,4.0,...,1,6,8,1,16,14,Morning,4663,6033.0,1.293802


### Scaling factor grouped by morning/afternoon/evening - concord

In [42]:
df2018_small.Study_Time.dtypes

dtype('O')

In [43]:
df2018_small['hour_transaction']=df2018_small.Study_Time.apply(lambda x: int(x.split(':')[0]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [44]:
df2018_small['parts_of_day'] = df2018_small['hour_transaction'].apply(assign_part_of_day)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [45]:
'''# Number of vehicles actually parked
actual_parking_load = sum(df2018['Total_Vehicle_Count'])
# Number of vehicles parked estimated by transaction data
estimated_parking_load = sum(df2018['Calculated Occupancy'])'''

"# Number of vehicles actually parked\nactual_parking_load = sum(df2018['Total_Vehicle_Count'])\n# Number of vehicles parked estimated by transaction data\nestimated_parking_load = sum(df2018['Calculated Occupancy'])"

In [0]:
df2018_small_scaling = pd.DataFrame(df2018_small.groupby(['parts_of_day'], as_index=False)[['Total_Vehicle_Count', 'Calculated Occupancy', 'Parking_Spaces']].sum())

In [0]:
df2018_small_scaling['scaling_factor'] = (df2018_small_scaling['Total_Vehicle_Count']/df2018_small_scaling['Calculated Occupancy'])

In [0]:
df2018_small_scaling['scaling_factor_trans_occ'] = (df2018_small_scaling['Total_Vehicle_Count'] - df2018_small_scaling['Calculated Occupancy'])/df2018_small_scaling['Parking_Spaces'] + 1

In [55]:
df2018_small_scaling

,parts_of_day,Total_Vehicle_Count,Calculated Occupancy,Parking_Spaces,scaling_factor,scaling_factor_trans_occ
0,Afternoon,4178.0,3631.0,4724.0,1.150647,1.115792
1,Evening,2159.0,1918.0,2703.0,1.125652,1.089160
2,Morning,2926.0,2332.0,3717.0,1.254717,1.159806
